In [ ]:
#import dependencies
from random import choice
import random
import heapq
import pandas as pd
import numpy as np
import math
from deap import base, creator, tools, algorithms
from plot_utils import *
from collections import defaultdict, deque, Counter
import time

#read data
position_data = pd.read_csv("sub_data_file_with_header.csv")
#position_data.head()

# #add the base station to the dataframe if not added
# new_rows = pd.DataFrame({'Nos.': [len(position_data) + 1, len(position_data) + 2],
#                          'x': [5000, -5000],
#                          'y': [-5000, 5000]})
# position_data = pd.concat([position_data, new_rows], ignore_index=True)

#extract the index, x, y
i = position_data.loc[:, 'No.']
x = position_data.loc[:,'x']
y = position_data.loc[:,'y']

#define function for distance and transmission rate

def distance(x1, x2, y1, y2):
    return math.sqrt(((x2 - x1) ** 2) + ((y2 - y1) ** 2))

def transmission_rate(d):
    if d >= 3000:
        return 0
    elif d >= 2500 and d <3000:
        return 1
    elif d >= 2000 and d < 2500:
        return 2
    elif d>= 1500 and d < 2000:
        return 3
    elif d >= 1000 and d < 1500:
        return 4
    elif d >= 500 and d < 1000:
        return 5
    elif d >0 and d< 500:
        return 7
    else:
        return 0

#generate the graph using the predefined functions
graph = {}
for r,x_1,y_1 in zip(i,x,y):
    graph["Node_" + str(r)] = {}
    for j, x_2,y_2 in zip(i,x,y):
        #if statement to chech if x1,y1,x2,y2 are the same, and continue
        if (x_1 == x_2) and (y_1 == y_2):
            continue
        #call the distance function, returns transmission rate
        dist = distance(x_1, x_2, y_1, y_2)
        tr_rate = transmission_rate(dist)
        #check if the transmission is greater than zero
        if tr_rate > 0:
            graph["Node_" + str(r)]["Node_" + str(j)] = tr_rate


#Helper functions
#Helps get transmission rate in a path
def get_tran_rate(bp):
    rates = []
    for i in range(len(bp)-1):
        rates.append(graph[bp[i]][bp[i+1]])
    return min(rates)

#Helps get row number to make plots easier
def extract_point(bp):
    all_value = []
    for i in bp:
        all_value.append(int(i.split("_")[1]) -1)
    return all_value

In [ ]:
#Hyperparameters
#max_trans_rate = 7
#max_latency = 1

def flatten_path(path):
    """Flatten a nested path into a single flat list."""
    flat_path = []
    for node in path:
        if isinstance(node, list):
            flat_path.extend(flatten_path(node))  # Recursively flatten nested lists
        else:
            flat_path.append(node)
    return flat_path

def cost_function(path):
    """
    Return (rate, latency).
    rate  = min transmission rate of all edges along this path
    latency = sum of 30 ms per link (or whichever rule you need).
    """
    if not path:
        return (0, 999999)  # Or handle empty path

    path = flatten_path(path)
    edge_rates = []
    for i in range(len(path) - 1):
        src, dst = path[i], path[i+1]
        if src in graph and dst in graph[src]:
            edge_rates.append(graph[src][dst])
        else:
            # No direct link => zero or "invalid" rate
            return (0, 999999)

    # Objective A: Maximize 'rate' => We take the minimum of all edges
    rate = min(edge_rates) if edge_rates else 0
    # Objective B: Minimize 'latency' => 30 ms per hop
    latency = (len(path) - 1) * 30
    return (rate, latency)

def fitness(rate, latency, alpha = 0.5, beta = 0.5):
    normalized_latency = latency / max_latency
    return alpha * rate - beta * normalized_latency

def parent(start, end, population):

  total_population = []
  while len(total_population) < population:
      visited = set()
      path = [start]
      visited.add(start)
      # print('path visited before while loop', path)
      # print('visited before while loop', visited)

      while path[-1] not in end:
          # Get unvisited neighbors only
          unvisited_neighbors = [node for node in graph[path[-1]] if node not in visited]

          # Break if no unvisited neighbors are left
          if not unvisited_neighbors:
              #print("No unvisited neighbors available, breaking the loop.")
              break

          # Choose a random unvisited neighbor
          next_choice = choice(unvisited_neighbors)
          if next_choice in end:
              path.append(next_choice)
              break
          # print('choice', next_choice)

          path.append(next_choice)
          visited.add(next_choice)
          # print('path visited after while loop', path)
          # print('visited after while loop', visited)

      if path[-1] in end and path not in total_population:
          total_population.append(path)


  return total_population


In [ ]:

# Number of generations and population size
generations = 50
population = 20
start = "Node_5"
end = ["Node_1", "Node_152"]



# Main genetic algorithm
def genetic_algorithm_with_tracking(start, end, population, generations):
    # **Initialization**: Create a random initial population of possible solutions (paths)
    parent_population = parent(start, end, population)

    # Initialize tracking lists
    best_latency_per_gen = []
    best_rate_per_gen = []
    avg_latency_per_gen = []
    avg_rate_per_gen = []

    for generation in range(generations):
        # **Selection**: Evaluate each solution in the current population using the cost function
        costs = [cost_function(path) for path in parent_population]  # Compute cost (rate, latency) for each path
        fitness_scores = [fitness(cost[0], cost[1]) for cost in costs]  # Evaluate fitness based on the cost

        # Combine paths with their fitness scores and sort by fitness (higher is better)
        population_with_scores = list(zip(parent_population, fitness_scores))
        sorted_population = sorted(population_with_scores, key=lambda x: x[1], reverse=True)

        # Select the top 50% of the population as parents for the next generation
        parent_population = [item[0] for item in sorted_population]
        top_parent = parent_population[:len(parent_population) // 2]

        # Track metrics for this generation
        latencies = [cost[1] for cost in costs]
        rates = [cost[0] for cost in costs]
        best_latency_per_gen.append(min(latencies))  # Best latency in the generation
        best_rate_per_gen.append(max(rates))        # Best rate in the generation
        avg_latency_per_gen.append(sum(latencies) / len(latencies))  # Average latency
        avg_rate_per_gen.append(sum(rates) / len(rates))            # Average rate

        # **Crossover**: Create new offspring by combining parts of two parent solutions
        offspring = []
        while len(offspring) < population - len(top_parent):
            # Randomly select two parents for crossover
            parent1, parent2 = random.sample(top_parent, 2)

            # Find a common crossover point in both parents
            common_nodes = set(parent1) & set(parent2) - {start}
            if common_nodes:
                # Perform crossover at the common node
                crossover_point = random.choice(list(common_nodes))
                idx1, idx2 = parent1.index(crossover_point), parent2.index(crossover_point)

                # Create two offspring by combining parts of both parents
                offspring1 = parent1[:idx1] + parent2[idx2:]
                offspring2 = parent2[:idx2] + parent1[idx1:]

                # Ensure the offspring are not identical to their parents
                parent_box = [parent1, parent2]
                if offspring1 not in parent_box:
                    child = offspring1
                else:
                    child = offspring2

                # Add valid offspring that reach the end node to the new population
                if child[-1] in end:
                    offspring.append(child)

        # Mutation: Introduce diversity in the offspring
        mutation_rate = 0.2  # Mutation occurs with a 20% probability
        for child in offspring:
            if len(child) > 2 and random.random() < mutation_rate:
                # Mutate by replacing a segment of the path with a new segment
                mutation_point = random.randint(1, len(child) - 2)
                new_segment = parent(child[mutation_point], end, 1)[0]
                if new_segment:
                    child[mutation_point:] = new_segment[0:]

        # Combine parents and offspring to form the new population
        parent_population = top_parent + offspring

    # Termination: Stop when the final generation is reached and return the best solution
    # Find the best path and its corresponding rate in the final population
    costs = [cost_function(path) for path in parent_population]
    fitness_scores = [fitness(cost[0], cost[1]) for cost in costs]
    best_index = fitness_scores.index(max(fitness_scores))  # Index of the best solution
    best_path = parent_population[best_index]  # Best path found
    best_rate, best_latency = cost_function(best_path)  # Best rate corresponding to the path

    return best_path, best_latency, best_rate, best_latency_per_gen, best_rate_per_gen, avg_latency_per_gen, avg_rate_per_gen


# # Run the genetic algorithm
best_path, best_latency, best_rate, _,_,_,_ = genetic_algorithm_with_tracking(start, end, population, generations)

# # Output the results
# print("Best Path:", best_path)  # Display the best path found
# print("Best Rate:", best_rate)  # Display the corresponding rate
# print("Best Rate:", best_latency)

Best Path: ['Node_5', 'Node_79', 'Node_78', 'Node_1']
Best Rate: 2
Best Rate: 90
